In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
import glob
from dataloader import *
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

### Dataloader testing

In [2]:
path = '../test_data/normal/'
num_workers = 1
batch_size=1
patchsize = (64,64)
margin = (80,80)

In [3]:
train_data = cevae(path,patchsize=patchsize,margin=margin)
trainLoader = DataLoader(train_data,batch_size=batch_size,num_workers=num_workers)

In [4]:
trainIter = iter(trainLoader)
test = trainIter.next()

In [5]:
for i,data in enumerate(trainLoader):
    print(data.shape)
    break

torch.Size([1, 1, 128, 128])


In [10]:
print(trainLoader.next())

AttributeError: 'DataLoader' object has no attribute 'next'

In [62]:
def random_bbox(image, margin, patchsize):
    """Generate a random tlhw with configuration.
        Args:
        config: Config should have configuration including IMG_SHAPES, VERTICAL_MARGIN, HEIGHT, HORIZONTAL_MARGIN, WIDTH.
        Returns:
        tuple: (top, left, height, width)
    """
    img_height = image.shape[0]
    img_width = image.shape[1]
    height = patchsize[0]
    width = patchsize[1]
    ver_margin = margin[0]
    hor_margin = margin[1]
    maxt = img_height - ver_margin - height
    maxl = img_width - hor_margin - width
    t = np.random.randint(low = ver_margin, high = maxt)
    l = np.random.randint(low = hor_margin, high = maxl)
    h = height
    w = width
    return (t, l, h, w)

def bbox2mask(image, margin, patchsize, times):
    """Generate mask tensor from bbox.
    Args:
    bbox: configuration tuple, (top, left, height, width)
    config: Config should have configuration including IMG_SHAPES,
    MAX_DELTA_HEIGHT, MAX_DELTA_WIDTH.
    Returns:
    tf.Tensor: output with shape [1, H, W, 1]
    """
    bboxs = []
    for i in range(times):
        bbox = random_bbox(image, margin, patchsize)
        bboxs.append(bbox)
        height = image.shape[0]
        width = image.shape[1]
        mask = np.zeros((height, width), np.float32)
        for bbox in bboxs:
            h = int(bbox[2] * 0.1) + np.random.randint(int(bbox[2] * 0.2 + 1))
            w = int(bbox[3] * 0.1) + np.random.randint(int(bbox[3] * 0.2) + 1)
            image[(bbox[0] + h) : (bbox[0] + bbox[2] - h), (bbox[1] + w) : (bbox[1] + bbox[3] - w)] = 1.
#             return image.reshape((1, ) + image.shape).astype(np.float32)
            return image

In [3]:
transform = transforms.Compose([transforms.ToPILImage(),transforms.RandomCrop(240),
                                transforms.RandomHorizontalFlip(),
                                transforms.Resize(128)])